In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import numpy as np
import plotly.graph_objects as go

In [8]:
data = pd.read_csv('data/Home/vsGKS.csv', delimiter=';')
data.head()

,Players,Ratings,Minutes,Goals,Assists,Offsides,Passes,Accurate_Passes,Accurate_Passes_%,Crosses,Tackles,Total_Shots,Shots_on_Target,Blocked_Shots,Yellow_Cards,Red_Cards,Fouls_Committed,Clearances,Saves
0,D. Abramowicz,"6,4",90,0,1,0,42,30,"71,43",2,1,2,0,0,1,0,1,4,0
1,S. Kerk,"6,3",83,0,0,0,27,21,"77,78",7,0,0,0,0,0,0,0,1,0
2,J. Oliveira,"6,1",16,0,0,0,0,0,"0,00",0,0,0,0,0,0,0,0,0,0
3,A. Nguiamba,"7,4",16,0,0,0,17,15,"88,24",0,2,0,0,0,0,0,0,0,0
4,D. Zator,"0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,00","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0"


In [9]:
data = data.set_index('Players')
data = data.replace(',', '.', regex=True)
data = data.apply(pd.to_numeric, errors='coerce')

In [10]:
data = data[data['Ratings'] != 0]

In [11]:
data.head()

,Ratings,Minutes,Goals,Assists,Offsides,Passes,Accurate_Passes,Accurate_Passes_%,Crosses,Tackles,Total_Shots,Shots_on_Target,Blocked_Shots,Yellow_Cards,Red_Cards,Fouls_Committed,Clearances,Saves
Players,,,,,,,,,,,,,,,,,,
D. Abramowicz,6.4,90.0,0.0,1.0,0.0,42.0,30.0,71.43,2.0,1.0,2.0,0.0,0.0,1.0,0.0,1.0,4.0,0.0
S. Kerk,6.3,83.0,0.0,0.0,0.0,27.0,21.0,77.78,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
J. Oliveira,6.1,16.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A. Nguiamba,7.4,16.0,0.0,0.0,0.0,17.0,15.0,88.24,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
K. Jakubczyk,5.8,74.0,0.0,0.0,0.0,19.0,17.0,89.47,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [12]:
# Create the figure
fig = go.Figure()

# List of metrics to plot
metrics = ['Ratings', 'Minutes', 'Goals', 'Assists', 'Offsides', 'Passes',
       'Accurate_Passes', 'Accurate_Passes_%', 'Crosses', 'Tackles',
       'Total_Shots', 'Shots_on_Target', 'Blocked_Shots', 'Yellow_Cards',
       'Red_Cards', 'Fouls_Committed', 'Clearances', 'Saves']

# Add traces for each metric
for metric in metrics:
    filtered_data = data[data[metrics] >  0]
    sorted_df = filtered_data.sort_values(by=metric, ascending=False)
    
    fig.add_trace(go.Bar(x=sorted_df.index, y=sorted_df[metric], name=metric))


# Define visibility matrix
n = len(metrics)
visibility_matrix = {
    "None": [False] * n,
    "AVG Rating": [metric in ['Ratings'] for metric in metrics],
    "Minutes": [metric in ['Minutes'] for metric in metrics],
    "Passes": [metric in ['Passes', 'Accurate_Passes', 'Accurate_Passes_%', 'Crosses'] for metric in metrics],
    "Attack": [metric in ['Goals', 'Assists', 'Total_Shots', 'Shots_on_Target', 'Shots_Inside_Box', 'Shots_Outside_Box'] for metric in metrics],
    "Defense": [metric in ['Tackles', 'Fouls_Drawn', 'Fouls_Committed', 'Saves'] for metric in metrics],
    "Cards": [metric in ['Yellow_Cards', 'Red_Cards'] for metric in metrics],
    "All": [True] * n
}

# Create buttons
buttons = [
    dict(label=label,
         method="update",
         args=[{"visible": visibility},
               {"title": f"Arka Match Stats: {label}"}])
    for label, visibility in visibility_matrix.items()
]

# Add dropdown to figure
fig.update_layout(
    updatemenus=[dict(
        active=0,
        buttons=buttons,
        x=1.1,
        y=1.15
    )],
    title="Arka Match Stats",
    xaxis_title="Match",
    yaxis_title="Value",
    barmode='group'
)

fig.show()
fig.write_html("arka_player_vs_gks_home.html")